In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

# (a) Download the AReM data from: https://archive.ics.uci.edu/ml/datasets/ Activity+Recognition+system+based+on+Multisensor+data+fusion+\%28AReM\ %29 . The dataset contains 7 folders that represent seven types of activities. In each folder, there are multiple files each of which represents an instant of a human performing an activity.1 Each file containis 6 time series collected from activitiesof the same person, which are called avg rss12, var rss12, avg rss13, var rss13, vg rss23, and ar rss23. There are 88 instances in the dataset, each of which con- tains 6 time series and each time series has 480 consecutive values.


> The corresponding data is downloaded and placed in the '../data' Directory 

# (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data.

In [2]:
# folders = ['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']
folders = {
    'bending1': 7,
    'bending2': 6,
    'cycling': 15,
    'lying' : 15,
    'sitting': 15,
    'standing': 15,
    'walking': 15  
}
testing_paths = []
training_paths = []
ordered_paths = []
for activity, _max in folders.items():
    for i in range(1, _max + 1):
        ordered_paths.append((f'../data/{activity}/dataset{i}.csv', activity))
        if i <= 2:
            testing_paths.append(f'../data/{activity}/dataset{i}.csv')    
        elif i == 3 and activity not in ['bending1', 'bending2']:
            testing_paths.append(f'../data/{activity}/dataset{i}.csv')   
        else:
            training_paths.append(f'../data/{activity}/dataset{i}.csv')
            
columns = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
testing_dfs = [pd.read_csv(path, skiprows = 4).iloc[:, 1:] for path in testing_paths]
training_dfs = [pd.read_csv(path, skiprows = 4).iloc[:, 1:] for path in training_paths]
ordered_dfs = [pd.read_csv(path, skiprows = 4).iloc[:, 1:] for path, activity in ordered_paths]
for i in range(len(ordered_dfs)):
    activity = ordered_paths[i][1]
    ordered_dfs[i]['Label'] = activity
print(f"Number of instances {len(testing_dfs) + len(training_dfs)}")
print(f"Number of testing instances {len(testing_dfs)}")
print(f"Number of training instances {len(training_dfs)}")

Number of instances 88
Number of testing instances 19
Number of training instances 69


# (c) Feature Extraction Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.

## i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

> Mean,
> Median, 
> Mode, 
> Standard Deviation,
> Variance,
> Covariance,
> Zero Cross Rate,
> Maximum value,
> Minimum value,
> Root Mean Square, 
> Distance feature = sqrt(sum of all the aforementioned metrics squared)

## ii. Extract the time-domain features minimum, maximum, mean, median, stan- dard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.2
    

In [3]:
cols = ["Instance", "Min(1)", "Max(1)", "Mean(1)", "Median(1)", "STD(1)", "Q1(1)", "Q3(1)",
           "Min(2)", "Max(2)", "Mean(2)", "Median(2)", "STD(2)", "Q1(2)", "Q3(2)",
           "Min(3)", "Max(3)", "Mean(3)", "Median(3)", "STD(3)", "Q1(3)", "Q3(3)",
           "Min(4)", "Max(4)", "Mean(4)", "Median(4)", "STD(4)", "Q1(4)", "Q3(4)",
           "Min(5)", "Max(5)", "Mean(5)", "Median(5)", "STD(5)", "Q1(5)", "Q3(5)",
           "Min(6)", "Max(6)", "Mean(6)", "Median(6)", "STD(6)", "Q1(6)", "Q3(6)",
           "Label"]
rows = []
for i, df in enumerate(ordered_dfs):
    temp = [i + 1]
    for col in df.columns[:-1]:
        temp = temp + [df[col].min(), df[col].max(), df[col].mean(), df[col].median(), df[col].std(), df[col].quantile(0.25), df[col].quantile(0.75)]
    temp.append(df.iloc[2,-1])
    rows.append(temp)
time_domain_df = pd.DataFrame(rows,columns = cols)

In [4]:
time_domain_df.head(10)

,Instance,Min(1),Max(1),Mean(1),Median(1),STD(1),Q1(1),Q3(1),Min(2),Max(2),...,Q1(5),Q3(5),Min(6),Max(6),Mean(6),Median(6),STD(6),Q1(6),Q3(6),Label
0,1,37.25,45.00,40.624792,40.50,1.476967,39.2500,42.00,0.0,1.30,...,33.0000,36.00,0.0,1.92,0.570583,0.43,0.582915,0.00,1.30,bending1
1,2,38.00,45.67,42.812812,42.50,1.435550,42.0000,43.67,0.0,1.22,...,32.0000,34.50,0.0,3.11,0.571083,0.43,0.601010,0.00,1.30,bending1
2,3,35.00,47.40,43.954500,44.33,1.558835,43.0000,45.00,0.0,1.70,...,35.3625,36.50,0.0,1.79,0.493292,0.43,0.513506,0.00,0.94,bending1
3,4,33.00,47.75,42.179813,43.50,3.670666,39.1500,45.00,0.0,3.00,...,30.4575,36.33,0.0,2.18,0.613521,0.50,0.524317,0.00,1.00,bending1
4,5,33.00,45.75,41.678063,41.75,2.243490,41.3300,42.75,0.0,2.83,...,28.4575,31.25,0.0,1.79,0.383292,0.43,0.389164,0.00,0.50,bending1
5,6,37.00,48.00,43.454958,43.25,1.386098,42.5000,45.00,0.0,1.58,...,22.2500,24.00,0.0,5.26,0.679646,0.50,0.622534,0.43,0.87,bending1
6,7,36.25,48.00,43.969125,44.50,1.618364,43.3100,44.67,0.0,1.50,...,20.5000,23.75,0.0,2.96,0.555313,0.49,0.487826,0.00,0.83,bending1
7,8,12.75,51.00,24.562958,24.25,3.737514,23.1875,26.50,0.0,6.87,...,20.5000,27.00,0.0,4.97,0.700188,0.50,0.693720,0.43,0.87,bending2
8,9,0.00,42.75,27.464604,28.00,3.583582,25.5000,30.00,0.0,7.76,...,15.0000,20.75,0.0,6.76,1.122125,0.83,1.012342,0.47,1.30,bending2
9,10,21.00,50.00,32.586208,33.00,6.238143,26.1875,34.50,0.0,9.90,...,17.6700,23.50,0.0,13.61,1.162042,0.83,1.332980,0.47,1.30,bending2


In [6]:
time_domain_df.tail(10)

,Instance,Min(1),Max(1),Mean(1),Median(1),STD(1),Q1(1),Q3(1),Min(2),Max(2),...,Q1(5),Q3(5),Min(6),Max(6),Mean(6),Median(6),STD(6),Q1(6),Q3(6),Label
78,79,21.50,51.00,34.935813,35.500,4.645944,32.0000,38.0625,0.0,12.21,...,14.2375,18.25,0.00,10.21,3.280021,3.015,1.700918,2.1200,4.5000,walking
79,80,18.33,47.67,34.333042,34.750,4.948770,31.2500,38.0000,0.0,12.48,...,13.7500,18.00,0.00,8.01,3.261583,2.980,1.617290,2.0500,4.3200,walking
80,81,18.33,45.75,34.599875,35.125,4.731790,31.5000,38.0000,0.0,15.37,...,14.0000,18.25,0.00,8.86,3.289542,3.015,1.680170,2.1200,4.2600,walking
81,82,15.50,43.67,34.225875,34.750,4.441798,31.2500,37.2500,0.0,17.24,...,14.3300,18.25,0.00,9.42,3.479542,3.270,1.761146,2.2400,4.5375,walking
82,83,21.50,51.25,34.253521,35.000,4.940741,30.9375,37.7500,0.0,13.55,...,13.7500,18.00,0.00,8.32,3.500750,3.285,1.692378,2.1800,4.5575,walking
83,84,19.50,45.33,33.586875,34.250,4.650935,30.2500,37.0000,0.0,14.67,...,13.7300,18.25,0.00,8.32,3.259729,3.110,1.640243,2.0500,4.3225,walking
84,85,19.75,45.50,34.322750,35.250,4.752477,31.0000,38.0000,0.0,13.47,...,13.5000,17.75,0.00,9.67,3.432562,3.200,1.732727,2.1575,4.5650,walking
85,86,19.50,46.00,34.546229,35.250,4.842294,31.2500,37.8125,0.0,12.47,...,14.0000,17.75,0.00,10.00,3.338125,3.080,1.656742,2.1600,4.3350,walking
86,87,23.50,46.25,34.873229,35.250,4.531720,31.7500,38.2500,0.0,14.82,...,13.7500,18.00,0.00,9.51,3.424646,3.270,1.690960,2.1700,4.5000,walking
87,88,19.25,44.00,34.473188,35.000,4.796705,31.2500,38.0000,0.0,13.86,...,13.7300,17.75,0.43,9.00,3.340458,3.090,1.699114,2.1200,4.3750,walking


## iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.